In [11]:
import cvxpy as cp
import scipy as sc
import numpy as np
import numpy.random as npr
import torch
from sklearn import datasets
import pandas as pd
import lropt
import sys
sys.path.append('..')
from utils import plot_tradeoff,plot_iters, plot_contours, plot_contours_line
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    
    "font.size":24,
    "font.family": "serif"
})

In [12]:
# Formulate constants
n = 2
N = 500
test_perc = 0.99
# k = npr.uniform(1,4,n)
# p = k + npr.uniform(2,5,n)
k_init = np.array([4.,5.])
p = np.array([5,6.5])
# k_tch = torch.tensor(k, requires_grad = True)?
# p_tch = torch.tensor(p, requires_grad = True)

def gen_demand_intro(N, seed):
    np.random.seed(seed)
    sig = np.array([[0.6,-0.3],[-0.3,0.1]])
    mu = np.array((1.1,1.7))
    norms = np.random.multivariate_normal(mu,sig, N)
    d_train = np.exp(norms)
    return d_train

def gen_demand_cor(N,seed,x):
    np.random.seed(seed)
    sig = np.eye(2)
    mu = np.array((6,7))
    points_list = []
    for i in range(N):
        mu_shift = -0.4*x[i]
        newpoint = np.random.multivariate_normal(mu+mu_shift,sig)
        points_list.append(newpoint)
    return np.vstack(points_list)

def calc_eval(x,p,k,u,t):
  val = 0
  vio = 0
  for i in range(u.shape[0]):
    val_cur = k[i]@x + np.max([-p[i][0]*x[0] - p[i][1]*x[1],-p[i][0]*x[0] - p[i][1]*u[i][1], -p[i][0]*u[i][0] - p[i][1]*x[1], -p[i][0]*u[i][0]- p[i][1]*u[i][1]]) 
    val+= val_cur
    vio += (val_cur >= t)
  return val/u.shape[0], vio/u.shape[0]
        
# Generate data
# data = gen_demand_intro(N, seed=5)
rawdata = gen_demand_intro(N, seed=18)

In [13]:
num_scenarios = N
num_reps = int(N/num_scenarios)
k_data = np.maximum(0.5,k_init + np.random.normal(0,3,(num_scenarios,n)))
p_data = k_data + np.maximum(0,np.random.normal(0,3,(num_scenarios,n)))
p_data = np.vstack([p_data]*num_reps)
k_data = np.vstack([k_data]*num_reps)

data = gen_demand_cor(N,seed=5,x=p_data)

In [15]:
test_p = 0.9
s = 8
np.random.seed(s)
# setup intial A, b
test_indices = np.random.choice(N,int(0.9*N), replace=False)
train_indices = [i for i in range(N) if i not in test_indices]
train = np.array([data[i] for i in train_indices])
test = np.array([data[i] for i in test_indices])
k_train = np.array([k_data[i] for i in train_indices])
k_test = np.array([k_data[i] for i in test_indices])
p_train = np.array([p_data[i] for i in train_indices])
p_test = np.array([p_data[i] for i in test_indices])

# Formulate uncertainty set
u = lropt.UncertainParameter(n,
                        uncertainty_set=lropt.Ellipsoidal(
                                                    data=data))
# Formulate the Robust Problem
x_r = cp.Variable(n)
t = cp.Variable()
k = cp.Parameter(n)
p = cp.Parameter(n)
p_x = cp.Variable(n)
objective = cp.Minimize(t)
constraints = []
for i in range(train.shape[0]):
    constraints += [-p_train[i][0]*x_r[0] - p_train[i][1]*x_r[1] + k_train[i]@x_r  <= t]
    constraints += [-p_train[i][0]*x_r[0] - p_train[i][1]*train[i][1] + k_train[i]@x_r  <= t]
    constraints += [ -p_train[i][0]*train[i][0] - p_train[i][1]*x_r[1]  + k_train[i]@x_r <= t]
    constraints+= [-p_train[i][0]*train[i][0]- p_train[i][1]*train[i][1]  + k_train[i]@x_r <= t]
constraints += [x_r >= 0]
prob = cp.Problem(objective, constraints)
prob.solve()
print(x_r.value)
eval, prob_vio = calc_eval(x_r.value, p_test, k_test,test,t.value)
print(eval,prob_vio)

[0.01425592 2.45593649]
-1.68234642446866 0.07777777777777778
